In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [2]:
import pandas as pd
import numpy as np

# Load datasets
train = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")
sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [3]:
train.info()
train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 18 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
 17  y          750000 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 103.0+ MB


id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [4]:
X = train.drop(columns=["id","y"])
y = train["y"]
X_test = test.drop(columns=["id"])

In [5]:
numeric_features = ["age","balance","day","duration","campaign","pdays","previous"]
ordinal_features = ["education","month"]
categorical_features = ["job","marital","default","housing","loan","contact","poutcome"]

print("Numeric:", numeric_features)
print("Ordinal:", ordinal_features)
print("Categorical:", categorical_features)

Numeric: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
Ordinal: ['education', 'month']
Categorical: ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'poutcome']


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [7]:
ordinal_mapping = {
    "education": ["unknown","primary","secondary","tertiary"],
    "month": ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
}

for col, categories in ordinal_mapping.items():
    encoder = OrdinalEncoder(categories=[categories])
    X[col] = encoder.fit_transform(X[[col]])
    X_test[col] = encoder.transform(X_test[[col]])


In [8]:
X = pd.get_dummies(X, columns=categorical_features)
X_test = pd.get_dummies(X_test, columns=categorical_features)

X, X_test = X.align(X_test, join="left", axis=1, fill_value=0)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [10]:
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=12, n_estimators=300, n_jobs=-1,
                       random_state=42)

In [11]:
y_val_pred = model.predict_proba(X_val)[:,1]
roc_auc = roc_auc_score(y_val, y_val_pred)
print("Validation ROC AUC:", roc_auc)

Validation ROC AUC: 0.9569839399397108


In [12]:
model.fit(X, y) 
y_test_pred = model.predict_proba(X_test)[:,1]

submission = pd.DataFrame({
    "id": test["id"],
    "y": y_test_pred
})

submission.to_csv("submission.csv", index=False)
print(" submission.csv ready for Kaggle!")

 submission.csv ready for Kaggle!
